In [1]:
# %load my_imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import datetime
import time
import re
import json

In [2]:
# Problem 1
pd.set_option('display.expand_frame_repr', False)

act_list = []
with open('movies/actor_movies.txt') as f:
    actors = f.readlines()
    
with open('movies/actress_movies.txt') as f:
    actresses = f.readlines()

In [3]:
actors[1]

'$, Homo\t\tNykytaiteen museo (1986)  \t\tSuuri illusioni (1985)  \n'

In [4]:
for i in range(6,15):
    print(actors[i].split('\t\t'))

['$torm, Cuntry', 'From the Woods: The Discovery of LYB (2015)  (as Country $torm)  \n']
['& Davi, Bruninho', '\n']
['& Dollar Furado, Caio Corsalette', '\n']
['& Fabiano, C\xe9sar Menotti', '\n']
['& Grand Piano Boogie Train, Jaap Dekker', '\n']
['& Ralph, Christian', '\n']
['& The Oriental Groove, Yacine', '\n']
['& Vin\xedcius, Jo\xe3o Bosco', '\n']
["'77 Big Smoker Pig", '\n']


In [5]:
movies = filter(None, re.split(r"[~\t\n]+", actors[6]))

In [6]:
movies

['$torm, Cuntry',
 'From the Woods: The Discovery of LYB (2015)  (as Country $torm)  ']

In [7]:
# Remove trailing spaces
map(lambda x: x.rstrip(), movies)

['$torm, Cuntry',
 'From the Woods: The Discovery of LYB (2015)  (as Country $torm)']

In [8]:
# Find out what's going on
er = 'Matsuzaki, Rei		Gekijou-ban Mahou Shoujo Madoka*Magica: 		Gekijou-ban Mahou Shoujo Madoka*Magica: 		Gekijouban Bureiku bureido Dairokushou: Doukoku no toride (2011)  (voice: Japanese version)  		Seiho redi: Kyonyû midara shibori (1999)  		Soulcalibur V (2012) (VG)  (v)'

In [9]:
er_filtered = filter(None, re.split(r"[~\t\n]+", er))
er_filtered

['Matsuzaki, Rei',
 'Gekijou-ban Mahou Shoujo Madoka*Magica: ',
 'Gekijou-ban Mahou Shoujo Madoka*Magica: ',
 'Gekijouban Bureiku bureido Dairokushou: Doukoku no toride (2011)  (voice: Japanese version)  ',
 'Seiho redi: Kyony\xc3\xbb midara shibori (1999)  ',
 'Soulcalibur V (2012) (VG)  (v)']

In [10]:
er_striped = map(lambda x: x.rstrip(), er_filtered)
er_striped

['Matsuzaki, Rei',
 'Gekijou-ban Mahou Shoujo Madoka*Magica:',
 'Gekijou-ban Mahou Shoujo Madoka*Magica:',
 'Gekijouban Bureiku bureido Dairokushou: Doukoku no toride (2011)  (voice: Japanese version)',
 'Seiho redi: Kyony\xc3\xbb midara shibori (1999)',
 'Soulcalibur V (2012) (VG)  (v)']

In [11]:
for name in er_striped[1:]:
    mov = re.split(r'[(|)]', name)
#     print (mov[0] + '(' + mov[1] + ')')
    print mov

['Gekijou-ban Mahou Shoujo Madoka*Magica:']
['Gekijou-ban Mahou Shoujo Madoka*Magica:']
['Gekijouban Bureiku bureido Dairokushou: Doukoku no toride ', '2011', '  ', 'voice: Japanese version', '']
['Seiho redi: Kyony\xc3\xbb midara shibori ', '1999', '']
['Soulcalibur V ', '2012', ' ', 'VG', '  ', 'v', '']


In [12]:
# perform a test run on actor[:50]
def clean_up_n_construct_dict(actor_list):
    actor_dict = {}
    for actor in actor_list:
        actor_movies = filter(None, re.split(r"[~\t\n]+", actor))
        actor = actor_movies[0]
        movies = map(lambda x: x.rstrip(), actor_movies[1:])
        # By observing the data, we explored movies with same name
        # adding set operation to eliminate such factor.
        if len(movies) >= 10:
            movies_clean = []
            for movie in movies:
                movie_str = re.split(r'[(|)]', movie)
                if len(movie_str) > 1:
                    movie_name = movie_str[0] + '(' + movie_str[1] + ')'
                    movies_clean.append(movie_name)
                else:
                    # now we find the bug, turns out 
                    # re.split(r'[(|)]', movie) return a list
                    # so if the length of the list is 1,
                    # we need to extract with movie_str[0]
                    movies_clean.append(movie_str[0])
            actor_dict.update({actor: movies_clean})
    return actor_dict

In [13]:
# Try
clean_up_n_construct_dict(actors[:50])

{'$hort, Too': ['American Pimp (1999)',
  'Beats, Rhymes & Life: The Travels of a Tribe Called Quest (2011)',
  'Gangsta Rap: The Glockumentary (2007)',
  'Ghetto Physics (2010)',
  'Mac Dre: Legend of the Bay (2014)',
  'Menace II Society (1993)',
  'Pimpalation: Return of the Trill (2006)',
  'Porndogs: The Adventures of Sadie (2009)',
  'Rhyme & Reason (1997)',
  'Stop Pepper Palmer (2014)',
  'Townbiz (2010)',
  'Uprising: Hip Hop and the LA Riots (2012)']}

In [14]:
# actors
actor_movie_dict = clean_up_n_construct_dict(actors)

In [15]:
len(actor_movie_dict)

74587

In [16]:
# merge actress
actor_movie_dict.update(clean_up_n_construct_dict(actresses))

In [17]:
len(actor_movie_dict)

113122

In [18]:
# Dict writer
import csv
def write_to_file(name, dict_object):
    with open(name, 'wb') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in dict_object.items():
           writer.writerow([key, value])     

In [19]:
actor_list = actor_movie_dict.keys()

In [20]:
len(actor_list)

113122

In [21]:
# Debugged hashable list issue
# simple n sweat 
# drop duplicates

movie_list = actor_movie_dict.values()
print('Actor length:',  len(movie_list))
# drop duplicates
# List Comprehension sooo Fast!!!
flatten = lambda x: [item for sublist in x for item in sublist]
movie_list = flatten(movie_list)
print('Movie length:', len(movie_list))

def drop_duplicates(seq):
    keys = {}
    for e in seq:
        keys[e] = 1
    return keys.keys()
movie_list = drop_duplicates(movie_list)
print('Movie length after dropping duplicates:', len(movie_list))

('Actor length:', 113122)
('Movie length:', 3176067)
('Movie length after dropping duplicates:', 468185)


In [22]:
# generate act_index_dict
# and movie_index_dict
# Such step is necessary since 
# writing files directly would take too much space
num_list = range(1, len(actor_list) + 1)
actor_index_dict = dict(zip(actor_list, num_list))

num_list = range(1, len(movie_list) + 1)
movie_index_dict = dict(zip(movie_list, num_list))

In [23]:
# write_to_file('actor_dict.csv', actor_index_dict)
# write_to_file('movie_dict.csv', movie_index_dict)

In [24]:
# Generate a actor_dict_int
# replace all str with numerics
# use deep copy to generate a new
# object
import copy
actor_movie_dict_int = copy.deepcopy(actor_movie_dict)
for old_key in actor_movie_dict_int.keys():
    new_key = actor_index_dict[old_key]
    actor_movie_dict_int[new_key] =actor_movie_dict_int.pop(old_key)
    
actor_movie_dict_int_int = copy.deepcopy(actor_movie_dict_int)

for key, values in actor_movie_dict_int_int.iteritems():
    for i in range(len(values)):
        actor_movie_dict_int_int[key][i] = movie_index_dict[values[i]] 

In [25]:
# write_to_file('actor_movie_dict_int_int.csv', actor_movie_dict_int_int)

In [26]:
# generate movie_actor_dict and
# movie_actor_dict_int_int
movie_actor_dict = {}
for movie in movie_list:
    movie_actor_dict.update({movie: []})

In [27]:
for key, values in actor_movie_dict.iteritems():
    for value in values:
        movie_actor_dict[value].append(key)

In [28]:
import copy
movie_actor_dict_int = copy.deepcopy(movie_actor_dict)
for old_key in movie_actor_dict_int.keys():
    new_key = movie_index_dict[old_key]
    movie_actor_dict_int[new_key] =movie_actor_dict_int.pop(old_key)
    
movie_actor_dict_int_int = copy.deepcopy(movie_actor_dict_int)

for key, values in movie_actor_dict_int_int.iteritems():
    for i in range(len(values)):
        movie_actor_dict_int_int[key][i] = actor_index_dict[values[i]] 

In [29]:
# write_to_file('movie_actor_dict_int_int.csv', movie_actor_dict_int_int)

In [30]:
## Construct trimmed movie_file
# movie edgelist
# remove movies with less than 10 actors
# 
movie_index_dict_trimmed = dict((y,x) for x, y in movie_index_dict.iteritems())
import copy
movie_actor_dict_int_int_trimmed = copy.deepcopy(movie_actor_dict_int_int)
movie_dict_trimmed = copy.deepcopy(movie_index_dict)
for movie, actors in movie_actor_dict_int_int.iteritems():
    if len(actors) < 10 :
        movie_actor_dict_int_int_trimmed.pop(movie)
        movie_index_dict_trimmed.pop(movie)
#     if len(actors) > 15:
#         movie_actor_dict_int_int_trimmed.pop(movie)
#         movie_index_dict_trimmed.pop(movie)
len(movie_actor_dict_int_int_trimmed)

105596

In [78]:
# Add three target movies in
target_movies = ['Batman v Superman: Dawn of Justice (2016)', 'Mission: Impossible - Rogue Nation (2015)', 'Minions (2015)']
for movie in target_movies:
    movie_actor_dict_int_int_trimmed.update({movie_index_dict[movie]: movie_actor_dict_int_int[movie_index_dict[movie]]})
    movie_index_dict_trimmed.update({movie_index_dict[movie]: movie})
write_to_file("trimmed_special/movie_actor_dict_int_int_trimmed.csv", movie_actor_dict_int_int_trimmed)
write_to_file("trimmed_special/movie_index_dict_trimmed.csv", movie_index_dict_trimmed)
len(movie_actor_dict_int_int_trimmed)

267804
[963, 2891, 3018, 3409, 5362, 5497, 5653, 5807, 7621, 15272, 17422, 17644, 17953, 18099, 24910, 29532, 30091, 30302, 30904, 32135, 32189, 34154, 35735, 40594, 41876, 44453, 46840, 47983, 49559, 52268, 59313, 60842, 61605, 62318, 62475, 62494, 64736, 66511, 67724, 68216, 70021, 70222, 72617, 75821, 76396, 82962, 83332, 84751, 85066, 85788, 87624, 88132, 88377, 88589, 90425, 94769, 96336, 96546, 97692, 98023, 99694, 105456, 107227, 110929, 111211, 111531]
415523
[3833, 6932, 8769, 12468, 14322, 16878, 18554, 20580, 24271, 24771, 25046, 27753, 28825, 31418, 34778, 35501, 39392, 42492, 46755, 49495, 51732, 53128, 53532, 54051, 54793, 55635, 58314, 58629, 59744, 62030, 63398, 64499, 64528, 66527, 66714, 66720, 67772, 69320, 69543, 72719, 75555, 78154, 82577, 82718, 85369, 86911, 87962, 88191, 88603, 90806, 90864, 91261, 93821, 95426, 95585, 96964, 97364, 99875, 103491, 108360, 109535, 111671]
289502
[7032, 23799, 31069, 33297, 42359, 43013, 49534, 52370, 59863, 64130, 64646, 67246, 7

52673

In [31]:
## Construct a trimmed index dict from it
# movie_index_dict_trimmed = dict((y,x) for x, y in movie_index_dict_trimmed.iteritems())
len(movie_index_dict_trimmed)

105596

In [32]:
# Rework
# Additional Section
# Also trim actor_list so as to speed up edge_list
# Instead of using original dict, construct a 
# actor_trimmed_dict from movie_trimmed_dict
actor_movie_dict_int_int_trimmed = {}
actor_list = movie_actor_dict_int_int_trimmed.values()
print('Actor length:',  len(actor_list))

# drop duplicates
flatten = lambda x: [item for sublist in x for item in sublist]
actor_list = flatten(actor_list)
print('Actor length:', len(actor_list))

def drop_duplicates(seq):
    keys = {}
    for e in seq:
        keys[e] = 1
    return keys.keys()
actor_list = drop_duplicates(actor_list)
print('Actor length after dropping duplicates:', len(actor_list))

('Actor length:', 105596)
('Actor length:', 1997378)
('Actor length after dropping duplicates:', 108912)


In [81]:
for actor in actor_list:
    actor_movie_dict_int_int_trimmed.update({actor: []})
    
for key, values in movie_actor_dict_int_int_trimmed.items():
    for value in values:
        actor_movie_dict_int_int_trimmed[value].append(key)

In [86]:
write_to_file('trimmed_special/actor_movie_dict_int_int_trimmed.csv', actor_movie_dict_int_int_trimmed)

In [83]:
actor_index_dict_reversed = dict((y,x) for x, y in actor_index_dict.iteritems())
actor_index_dict_trimmed = {}
for actor in actor_movie_dict_int_int_trimmed.keys():
    actor_index_dict_trimmed.update({actor_index_dict_reversed[actor]: actor})

In [84]:
print (len(actor_movie_dict_int_int_trimmed))
print (len(actor_index_dict_trimmed))

100707
100707


In [85]:
write_to_file("trimmed_special/actor_index_dict_trimmed.csv", actor_index_dict_trimmed)